In [1]:
setwd('/Users/alexis/University of North Carolina at Chapel Hill/Rager, Julia - CEMALB_DataAnalysisPM/Projects/P1002. Ecig Metabolomics NELF/P1002.2. Original Data')
Output = ('/Users/alexis/University of North Carolina at Chapel Hill/Rager, Julia - CEMALB_DataAnalysisPM/Projects/P1002. Ecig Metabolomics NELF/P1002.3. Analyses/P1002.3.1. Smoking Status Prediction/Output')
cur_date = "020822"

library(readxl)
library(tidyverse)
library(gtsummary)
library(caret)
library(e1071)
library(Hmisc)
library(glmnet)

#reading in file
molecular_feature_df = data.frame(read_excel("020722_mf_qrilc_df_square.xlsx")) %>%
    rename(Smoking_Status = group)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.0.6     ✔ dplyr   1.0.4
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift


Loading required package: survival


Attaching package: ‘survival’


The following object is masked from ‘package:caret’:

    cluster


Loading required package: Formula


Attaching package: ‘Hmisc’


The following object is masked from ‘package:e1071’:

    impute


The following objects are masked from ‘package:dplyr’:

    src, summarize


The following objects are masked from ‘package:base’:

    format.pval, units


Loading required package: Matrix


Attaching package: ‘Mat

In [2]:
head(molecular_feature_df)

,sample,Smoking_Status,MF_2,MF_3,MF_4,MF_5,MF_6,MF_7,MF_11,MF_12,⋯,MF_62282,MF_62598,MF_62719,MF_62853,MF_63431,MF_64025,MF_64936,MF_68286,MF_77165,MF_78976
,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,70,NS,20.06575,18.04776,19.41787,18.23115,17.64608,18.55202,18.63432,19.51753,⋯,19.04825,17.84468,17.05958,19.27718,19.30373,17.50497,17.97432,19.34839,17.93527,17.49503
2,110,NS,21.29010,22.79702,21.25771,17.10627,16.42912,20.62507,20.59117,21.33268,⋯,19.53205,18.96149,18.80578,17.98273,17.34886,18.13338,19.27253,19.60386,19.47317,19.50360
3,143,NS,13.32488,14.65592,13.67219,17.84354,16.72274,17.22157,17.33919,13.99090,⋯,16.67609,19.76019,16.29403,19.34838,19.22524,15.75499,17.33830,17.53654,19.75012,19.74391
4,152,NS,21.80749,23.43693,22.19403,17.00609,17.79543,21.74936,21.36163,22.28248,⋯,19.27962,19.53391,19.71066,19.49494,19.74998,15.90220,19.68467,16.68921,19.48784,19.52267
5,187,NS,19.64273,18.09695,19.89369,17.51687,17.15478,18.07787,16.42266,19.84661,⋯,19.35298,20.07743,19.65564,16.88768,17.23001,17.32652,18.71168,19.63040,19.55652,19.15030
6,196,NS,22.73848,24.57595,22.97987,16.02619,16.89026,22.76940,17.36809,23.00538,⋯,19.41148,19.72106,19.91198,17.55342,17.67836,15.80386,15.76951,18.23732,16.94863,17.49168


In [3]:
#first making smoking status column binary denoted by 0 = non smoker and 1 = ecig user
#removing sample col which isn't necessary
molecular_feature_df = molecular_feature_df[2:3052] %>%
    #grepl is synonymous with "contains"
    #necessary to put it into factor and relevel, so algorithm understands which data is considered baseline
    mutate(Smoking_Status = relevel(factor(ifelse(grepl("ECig", Smoking_Status), 1, 0)), ref = "0"))

head(molecular_feature_df)

,Smoking_Status,MF_2,MF_3,MF_4,MF_5,MF_6,MF_7,MF_11,MF_12,MF_13,⋯,MF_62282,MF_62598,MF_62719,MF_62853,MF_63431,MF_64025,MF_64936,MF_68286,MF_77165,MF_78976
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0,20.06575,18.04776,19.41787,18.23115,17.64608,18.55202,18.63432,19.51753,19.67167,⋯,19.04825,17.84468,17.05958,19.27718,19.30373,17.50497,17.97432,19.34839,17.93527,17.49503
2,0,21.29010,22.79702,21.25771,17.10627,16.42912,20.62507,20.59117,21.33268,21.21878,⋯,19.53205,18.96149,18.80578,17.98273,17.34886,18.13338,19.27253,19.60386,19.47317,19.50360
3,0,13.32488,14.65592,13.67219,17.84354,16.72274,17.22157,17.33919,13.99090,15.39366,⋯,16.67609,19.76019,16.29403,19.34838,19.22524,15.75499,17.33830,17.53654,19.75012,19.74391
4,0,21.80749,23.43693,22.19403,17.00609,17.79543,21.74936,21.36163,22.28248,22.17892,⋯,19.27962,19.53391,19.71066,19.49494,19.74998,15.90220,19.68467,16.68921,19.48784,19.52267
5,0,19.64273,18.09695,19.89369,17.51687,17.15478,18.07787,16.42266,19.84661,19.52623,⋯,19.35298,20.07743,19.65564,16.88768,17.23001,17.32652,18.71168,19.63040,19.55652,19.15030
6,0,22.73848,24.57595,22.97987,16.02619,16.89026,22.76940,17.36809,23.00538,22.29729,⋯,19.41148,19.72106,19.91198,17.55342,17.67836,15.80386,15.76951,18.23732,16.94863,17.49168


The goal of this analysis is to use molecular features [continuous] to predict smoking staus (dichotomous). This will be done using Support Vector Machine(SVM) based on a logistic regression model for classification. It was also chosen since SVM does a better job at handling a large number of predictors since p > n. 

# Support Vector Machine
- supervised learning models that can predict continuous (regression) or grouped/dichotomous (classification) outcomes
- predictors by projecting them onto a high dimensional space and uses kernels to make the data more separable (unfortunately makes interpretability of model results more difficult)

In [5]:
#model errors are calculated to assess the predictive accuracy of the model
svm_classification_model_errors = function(dataset, model, outcome, pred_outcome, gamma_values, cost_values, 
                                           elsilon_values){
  
    #setting seed for reproducibility
    set.seed(12)
    
    #splitting data into training and testing sets
    dataset_index = createFolds(dataset[[outcome]], k = 5) #5 fold CV
    errors = data.frame()
    for (i in 1:length(dataset_index)){
        data_train = dataset[-dataset_index[[i]],]
        data_test = dataset[dataset_index[[i]],]

        #now pruning parameters (based on the training dataset to prevent overfitting)
        svr_tune <- tune(svm, as.formula(paste0(outcome, "~.")), data = data_train, kernel = model, 
                       ranges = list(elsilon = elsilon_values, cost = cost_values, gamma = gamma_values)) 

        #choosing best model
        best_svm <- svr_tune$best.model

        #predicting with tuned parameters 
        data_test[[pred_outcome]] <- predict(best_svm, newdata = data_test, type = "response")

        matrix = confusionMatrix(data = data_test[[pred_outcome]], reference = data_test[[outcome]], 
                                 positive = "1")
        #extracting accuracy, sens, spec, PPV, NPV to take mean and sd later
        error_values = data.frame(t(c(matrix$overall[1], matrix$byClass[1:4])))

        #adding values to df
        errors = rbind(errors, error_values)
  }
  
  #taking averages/sd 
  errors = errors %>%
    summarise(#`Accuracy CV Error` = mean(Accuracy), 
            `Sensitivity CV Error` = mean(Sensitivity), `Specificity CV Error` = mean(Specificity), 
              #`PPV CV Error` = mean(Pos.Pred.Value), `NPV CV Error` = mean(Neg.Pred.Value), 
             # `Accuracy Std Error` = sd(Accuracy),
              `Sensitivity Std Error` = sd(Sensitivity), `Specificity Std Error` = sd(Specificity),
              #`PPV CV Error` = sd(Pos.Pred.Value), `NPV CV Error` = sd(Neg.Pred.Value)
             )
  
  return(errors)
}

#calling fn
#linear kernel
svm_linear = svm_classification_model_errors(molecular_feature_df, "linear", "Smoking_Status", "pred_Smoking_Status",
                                            0.035, 1:5, seq(0,1,0.2))

In [6]:
#radial
svm_radial = svm_classification_model_errors(molecular_feature_df, "radial", "Smoking_Status", "pred_Smoking_Status",
                                             0.035, 1:5, seq(0,1,0.2))
#polynomial
#elison value is fixed at 0.1, but I left it so I didn't have to create a new function
svm_polynomial = svm_classification_model_errors(molecular_feature_df, "polynomial", "Smoking_Status", 
                                             "pred_Smoking_Status", 0.035, 1:5, 0.1)

In [7]:
#creating one combined df
final_df = data.frame(rbind(svm_linear, svm_radial, svm_polynomial))
final_df = data.frame(Kernel = c("linear", "radial", "polynomial"), final_df[,1:4])
final_df

Kernel,Sensitivity.CV.Error,Specificity.CV.Error,Sensitivity.Std.Error,Specificity.Std.Error
<fct>,<dbl>,<dbl>,<dbl>,<dbl>
linear,0.6809524,0.6400000,0.2100103,0.3443190
radial,1.0000000,0.0000000,0.0000000,0.0000000
polynomial,0.8714286,0.2333333,0.1384233,0.2211083


> The polynomial kernel performed the best overall. 

Determing if there are highly correlated variables that can be removed.

In [8]:
#creating correlation matrix
#the cut off can be arbitrary, but due to the high number of correlations, 0.85 became the cutoff
corr_matrix = rcorr(as.matrix(molecular_feature_df[2:3051]), type = "spearman")
#creating df where I can easily view coefficents that are highly correlated (rho > 0.7 or rho < -0.7)
corr_matrix_df = data.frame(corr_matrix$r) %>%
  rownames_to_column(var = "Variable1") %>%
  #needed to specificy package of melt for it to work
  reshape2::melt(variable = "Variable2", value.name = "Spearman Coefficient") %>%
  #also removed values = 1, because those were correlations between the same variable
  filter(abs(`Spearman Coefficient`) < 1 & abs(`Spearman Coefficient`) > 0.7) %>%
  arrange(desc(`Spearman Coefficient`)) %>%
  #removing duplicate spearman coefficient values and keeping all columns
  distinct(`Spearman Coefficient`, .keep_all = TRUE) 

head(corr_matrix_df)

Using Variable1 as id variables



,Variable1,Variable2,Spearman Coefficient
,<chr>,<fct>,<dbl>
1,MF_586,MF_7,0.9989480
2,MF_124,MF_34,0.9987142
3,MF_516,MF_7,0.9974868
4,MF_301,MF_133,0.9973115
5,MF_54,MF_4,0.9969024
6,MF_586,MF_516,0.9965517


In [24]:
#determining how many times each feature occurs
correlation_occurences1 = corr_matrix_df %>%
    group_by(Variable1) %>%
    count() %>%
    arrange(-n) %>%
    filter(n > 20)

#correlation_occurences1

correlation_occurences2 = corr_matrix_df %>%
    group_by(Variable2) %>%
    count() %>%
    arrange(-n) %>%
    filter(n > 20)

#correlation_occurences2

all_corr_occurences = unique(c(correlation_occurences1$Variable1, as.character(correlation_occurences2$Variable2)))

In [25]:
all_corr_occurences

[1] "MF_5984"  "MF_3566"  "MF_2827"  "MF_3296"  "MF_12432" "MF_3514" 
 [7] "MF_1537"  "MF_3067"  "MF_4881"  "MF_6815"  "MF_1615"  "MF_1734" 
[13] "MF_1065"  "MF_1391"  "MF_1431"  "MF_1945"  "MF_9940"  "MF_1118" 
[19] "MF_1494"  "MF_5086"  "MF_660"   "MF_1138"  "MF_1221"  "MF_2083" 
[25] "MF_644"   "MF_7167"  "MF_17636" "MF_2297"  "MF_4576"  "MF_6259" 
[31] "MF_4"     "MF_7"     "MF_2"     "MF_54"    "MF_93"    "MF_3"    
[37] "MF_277"   "MF_316"   "MF_48"    "MF_154"   "MF_516"   "MF_75"   
[43] "MF_34"    "MF_586"   "MF_29"    "MF_91"    "MF_156"   "MF_477"  
[49] "MF_15"    "MF_23"    "MF_169"   "MF_317"   "MF_18"    "MF_200"  
[55] "MF_566"   "MF_747"   "MF_73"    "MF_289"   "MF_426"   "MF_230"  
[61] "MF_342"   "MF_167"   "MF_331"   "MF_163"   "MF_260"   "MF_343"  
[67] "MF_570"   "MF_124"   "MF_412"   "MF_133"

In [26]:
#removing molecular features (MF) that correlate most often
`%notin%` <- Negate(`%in%`)
penalized_dataset = molecular_feature_df[,colnames(molecular_feature_df) %notin% all_corr_occurences]

In [27]:
#calling fn
#linear kernel
svm_linear = svm_classification_model_errors(penalized_dataset, "linear", "Smoking_Status", "pred_Smoking_Status",
                                             0.035, 1:5, seq(0,1,0.2))
#radial
svm_radial = svm_classification_model_errors(penalized_dataset, "radial", "Smoking_Status", "pred_Smoking_Status",
                                             0.035, 1:5, seq(0,1,0.2))
#polynomial
#elison value is fixed at 0.1, but I left it so I didn't have to create a new function
svm_polynomial = svm_classification_model_errors(penalized_dataset, "polynomial", "Smoking_Status", 
                                             "pred_Smoking_Status", 0.035, 1:5, 0.1)


#creating one combined df
penalized_final_df = data.frame(rbind(svm_linear, svm_radial, svm_polynomial))
penalized_final_df = data.frame(Kernel = c("linear", "radial", "polynomial"), penalized_final_df[,1:4])
penalized_final_df

Kernel,Sensitivity.CV.Error,Specificity.CV.Error,Sensitivity.Std.Error,Specificity.Std.Error
<fct>,<dbl>,<dbl>,<dbl>,<dbl>
linear,0.6809524,0.6400000,0.2100103,0.3443190
radial,1.0000000,0.0000000,0.0000000,0.0000000
polynomial,0.8428571,0.3066667,0.1557660,0.3003701


In [28]:
final_df
penalized_final_df

Kernel,Sensitivity.CV.Error,Specificity.CV.Error,Sensitivity.Std.Error,Specificity.Std.Error
<fct>,<dbl>,<dbl>,<dbl>,<dbl>
linear,0.6809524,0.6400000,0.2100103,0.3443190
radial,1.0000000,0.0000000,0.0000000,0.0000000
polynomial,0.8714286,0.2333333,0.1384233,0.2211083


Kernel,Sensitivity.CV.Error,Specificity.CV.Error,Sensitivity.Std.Error,Specificity.Std.Error
<fct>,<dbl>,<dbl>,<dbl>,<dbl>
linear,0.6809524,0.6400000,0.2100103,0.3443190
radial,1.0000000,0.0000000,0.0000000,0.0000000
polynomial,0.8428571,0.3066667,0.1557660,0.3003701
